In [3]:
from datasets import load_dataset
import json

In [16]:
from datasets import load_dataset

qasper = load_dataset("allenai/qasper")

print("🔍 Dataset loaded successfully.")
print("Train size:", len(qasper["train"]))
print("Validation size:", len(qasper["validation"]))
print("Test size:", len(qasper["test"]))


🔍 Dataset loaded successfully.
Train size: 888
Validation size: 281
Test size: 416


In [17]:
print("\n🧾 Raw entry from train split:")
print(qasper["train"][0])



🧾 Raw entry from train split:
{'id': '1909.00694', 'title': 'Minimally Supervised Learning of Affective Events Using Discourse Relations', 'abstract': 'Recognizing affective events that trigger positive or negative sentiment has a wide range of natural language processing applications but remains a challenging problem mainly because the polarity of an event is not necessarily predictable from its constituent words. In this paper, we propose to propagate affective polarity using discourse relations. Our method is simple and only requires a very small seed lexicon and a large raw corpus. Our experiments using Japanese data show that our method learns affective events effectively without manually labeled data. It also improves supervised learning results when labeled data are small.', 'full_text': {'section_name': ['Introduction', 'Related Work', 'Proposed Method', 'Proposed Method ::: Polarity Function', 'Proposed Method ::: Discourse Relation-Based Event Pairs', 'Proposed Method ::: Di

In [22]:
train_data = basic_debug_extractor(qasper["train"], "train")
with open("train.json", "w") as f:
    json.dump(train_data, f, indent=2)


🔍 [train] Processing QA #1 (Item 0, QA 0)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #2 (Item 0, QA 1)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #3 (Item 0, QA 2)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #4 (Item 0, QA 3)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #5 (Item 0, QA 4)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #6 (Item 0, QA 5)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #7 (Item 0, QA 6)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #8 (Item 0, QA 7)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #9 (Item 1, QA 0)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #10 (Item 1, QA 1)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #11 (Item 1, QA 2)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #12 (Item 1, QA 3)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #13 (Item 1, QA 4)
⛔ Skipped: QA is not a dict.

🔍 [train] Processing QA #14 (Item 1, QA 5)
⛔ Skipped: QA is

In [21]:
def basic_debug_extractor(split_data, split_name, limit_debug=3):
    results = []
    printed = 0
    total = 0
    skipped = 0

    for item_idx, item in enumerate(split_data):
        abstract = item.get("abstract", "")
        if not abstract:
            continue

        qas = item.get("qas", [])
        for qa_idx, qa in enumerate(qas):
            total += 1

            print(f"\n🔍 [{split_name}] Processing QA #{total} (Item {item_idx}, QA {qa_idx})")

            # Check type
            if not isinstance(qa, dict):
                print("⛔ Skipped: QA is not a dict.")
                skipped += 1
                continue

            question = qa.get("question", "")
            answers = qa.get("answers", [])

            # Check for valid question and answers
            if not question:
                print("⛔ Skipped: Missing question.")
                skipped += 1
                continue

            if not answers or not isinstance(answers[0], dict):
                print("⛔ Skipped: Answers missing or malformed.")
                skipped += 1
                continue

            answer = answers[0].get("answer", "")
            if not answer:
                print("⛔ Skipped: No answer text.")
                skipped += 1
                continue

            # Build the instruction-style entry
            input_text = f"{abstract.strip()}\nQuestion: {question.strip()}"
            output_text = answer.strip()

            results.append({
                "input": input_text,
                "output": output_text
            })

            if printed < limit_debug:
                print("✅ Sample entry added:")
                print("Input:", input_text[:300], "...")
                print("Output:", output_text)
                printed += 1

    print(f"\n📊 [{split_name}] Summary:")
    print(f"  Total QA pairs processed: {total}")
    print(f"  Skipped: {skipped}")
    print(f"  Saved: {len(results)}")

    return results


In [ ]:
from datasets import load_dataset
import json

def extract_qasper_entry_format(split_data, split_name, limit_debug=3):

    results = []
    printed = 0
    total = 0
    skipped = 0

    for item_idx, item in enumerate(split_data):
        print(f"\n🧪 RAW EXAMPLE FROM SPLIT [{split_name}]:\n")
        print(json.dumps(item, indent=2))
        break
        abstract = item.get("abstract", "").strip()
        if not abstract or "question" not in item:
            continue

        questions = item["question"]
        question_ids = item["question_id"]
        answers_list = item["answers"]

        for i in range(len(questions)):
            total += 1

            question = questions[i].strip()
            answer_block = answers_list[i]

            # Check if 'answer' exists and is a list
            if "answer" not in answer_block or not answer_block["answer"]:
                skipped += 1
                continue

            answer_entry = answer_block["answer"][0]  # Take first answer

            if answer_entry.get("unanswerable", False):
                skipped += 1
                continue

            # Prefer free_form_answer, fallback to extractive span
            answer = answer_entry.get("free_form_answer", "").strip()
            if not answer:
                spans = answer_entry.get("extractive_spans", [])
                if spans and isinstance(spans[0], str):
                    answer = spans[0].strip()

            if not answer:
                skipped += 1
                continue

            explanation = answer_entry.get("highlighted_evidence", [])
            if explanation and isinstance(explanation, list):
                explanation = explanation[0]
            else:
                explanation = ""

            input_text = f"{abstract}\nQuestion: {question}"
            output_text = f"{answer}. Explanation: {explanation}" if explanation else answer

            results.append({"input": input_text, "output": output_text})

            if printed < limit_debug:
                print(f"\n✅ [{split_name}] Example #{printed + 1}")
                print("Input:", input_text[:300], "...")
                print("Output:", output_text)
                printed += 1

    print(f"\n📊 [{split_name}] Summary:")
    print(f"  Total QA pairs processed: {total}")
    print(f"  Skipped: {skipped}")
    print(f"  Saved: {len(results)}")

    return results


# Run and save
qasper = load_dataset("allenai/qasper")

for split_name in ["train", "validation", "test"]:
    split_data = qasper[split_name]
    results = extract_qasper_entry_format(split_data, split_name)

    with open(f"{split_name}.json", "w") as f:
        json.dump(results, f, indent=2)

    print(f"💾 {split_name}.json written ✅\n")


SyntaxError: 'break' outside loop (2868382252.py, line 7)

In [13]:
def extract_instruction_format(split_data, split_name):
    results = []
    skipped = 0

    for item in split_data:
        abstract = item.get("abstract", "")
        if not abstract:
            continue

        qas = item.get("qas", [])
        for qa in qas:
            if not isinstance(qa, dict):
                skipped += 1
                continue

            question = qa.get("question", "")
            answers = qa.get("answers", [])

            if not (question and answers and isinstance(answers[0], dict)):
                skipped += 1
                continue

            # Extract answer
            raw_answer = answers[0].get("answer", "")
            raw_expl = answers[0].get("explanation", "")

            if isinstance(raw_answer, list):
                answer = next((a for a in raw_answer if isinstance(a, str) and a.strip()), "")
            elif isinstance(raw_answer, str):
                answer = raw_answer.strip()
            else:
                answer = ""

            if isinstance(raw_expl, list):
                explanation = next((e for e in raw_expl if isinstance(e, str) and e.strip()), "")
            elif isinstance(raw_expl, str):
                explanation = raw_expl.strip()
            else:
                explanation = ""

            if not (abstract.strip() and question.strip() and answer):
                skipped += 1
                continue

            input_text = f"{abstract.strip()}\nQuestion: {question.strip()}"
            output_text = f"{answer}. Explanation: {explanation}" if explanation else answer

            results.append({
                "input": input_text,
                "output": output_text
            })

    if results:
        print(f"✅ {split_name}: {len(results)} examples | Skipped: {skipped}")
        print("🧪 Sample entry:")
        print(json.dumps(results[0], indent=2))
    else:
        print(f"⚠️ {split_name}: No valid entries found.")

    return results


In [14]:
# Load dataset
qasper = load_dataset("allenai/qasper")
splits = {
    "train": qasper["train"],
    "validation": qasper["validation"],
    "test": qasper["test"]
}

In [15]:
# Process and save each split
for split_name, split_data in splits.items():
    formatted = extract_instruction_format(split_data, split_name)
    with open(f"{split_name}.json", "w") as f:
        json.dump(formatted, f, indent=2)
    print(f"💾 Saved {split_name}.json\n")

⚠️ train: No valid entries found.
💾 Saved train.json

⚠️ validation: No valid entries found.
💾 Saved validation.json

⚠️ test: No valid entries found.
💾 Saved test.json



In [7]:
print(f"🧪 Sample {split_name} example:\n")
print(json.dumps(results[0], indent=2))


NameError: name 'split_name' is not defined

In [8]:
# Load splits
qasper = load_dataset("allenai/qasper")
splits = {
    "train": qasper["train"],
    "validation": qasper["validation"],
    "test": qasper["test"]
}

In [9]:
formatted = extract_instruction_format(split_data)

if formatted:
    print(f"🧪 Sample from {split_name}:")
    print(json.dumps(formatted[0], indent=2))
else:
    print(f"⚠️ No usable entries found in {split_name} split.")

with open(f"{split_name}.json", "w") as f:
    json.dump(formatted, f, indent=2)

print(f"✅ Saved {len(formatted)} examples to {split_name}.json")


NameError: name 'split_data' is not defined